In [262]:
import pandas as pd
import ast
import numpy as np

In [263]:
path = 'neutral.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [264]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [265]:
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [266]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,If you catch your girlfriend lying about seein...,"[broken heart, mistrust, getting dumped, being...",Questions: If you catch your girlfriend lying ...,1. The correct answer is **(B) mistrust**. If ...,1. the correct answer is **(b) mistrust**. if ...


In [267]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [268]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [269]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [290]:
# read paper's answer
paper = pd.read_csv('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] CQA/paper.csv', index_col=0)

In [291]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)

In [292]:
df.set_index('premise', inplace=True)

KeyError: "None of ['premise'] are in the columns"

In [293]:
paper['my_label'] = df['LLM_answer']

In [294]:
sum(paper.label == paper.my_label) / len(paper)

0.8181911508058721

In [296]:
df.to_csv('neutral - full.csv')